In [4]:
import sys
dataDir = '../../VQA'
sys.path.insert(0, '%s/PythonHelperTools/vqaTools' %(dataDir))

In [5]:
from vqa import VQA
from vqaEvaluation.vqaEval import VQAEval
import matplotlib.pyplot as plt
import skimage.io as io
import json
import random
import os

In [6]:
# set up file names and paths
versionType ='v2_' # this should be '' when using VQA v2.0 dataset
taskType    ='OpenEnded' # 'OpenEnded' only for v2.0. 'OpenEnded' or 'MultipleChoice' for v1.0
dataType    ='mscoco'  # 'mscoco' only for v1.0. 'mscoco' for real and 'abstract_v002' for abstract for v1.0. 
dataSubType ='val2014'
annFile     ='%s/Annotations/%s%s_%s_annotations.json'%(dataDir, versionType, dataType, dataSubType)
quesFile    ='%s/Questions/%s%s_%s_%s_questions.json'%(dataDir, versionType, taskType, dataType, dataSubType)
imgDir      ='%s/Images/%s/%s/' %(dataDir, dataType, dataSubType)
resultType  ='fake'
fileTypes   = ['results', 'accuracy', 'evalQA', 'evalQuesType', 'evalAnsType'] 

In [7]:
[resFile, accuracyFile, evalQAFile, evalQuesTypeFile, evalAnsTypeFile] = ['%s/Results/%s%s_%s_%s_%s_%s.json'%(dataDir, versionType, taskType, dataType, dataSubType, \
resultType, fileType) for fileType in fileTypes]  

In [8]:
vqa = VQA(annFile, quesFile)
vqaRes = vqa.loadRes(resFile, quesFile)

loading VQA annotations and questions into memory...
0:00:07.236862
creating index...
index created!
Loading and preparing results...     
DONE (t=0.48s)
creating index...
index created!


In [9]:
vqaEval = VQAEval(vqa, vqaRes, n=2)

In [10]:
"""
If you have a list of question ids on which you would like to evaluate your results, pass it as a list to below function
By default it uses all the question ids in annotation file
"""
vqaEval.evaluate() 

computing accuracy
Finshed Percent: [####################] 99% Done computing accuracy


In [11]:
# print accuracies
print ("\n")
print ("Overall Accuracy is: %.02f\n" %(vqaEval.accuracy['overall']))
print ("Per Question Type Accuracy is the following:")
for quesType in vqaEval.accuracy['perQuestionType']:
	print ("%s : %.02f" %(quesType, vqaEval.accuracy['perQuestionType'][quesType]))
print ("\n")
print ("Per Answer Type Accuracy is the following:")
for ansType in vqaEval.accuracy['perAnswerType']:
	print ("%s : %.02f" %(ansType, vqaEval.accuracy['perAnswerType'][ansType]))
print ("\n")



Overall Accuracy is: 84.73

Per Question Type Accuracy is the following:
none of the above : 81.84
what are the : 79.57
what is : 78.98
what : 76.71
is this a : 98.79
is this : 98.24
what is the man : 84.00
how many : 72.06
what does the : 72.18
why : 35.21
is it : 98.72
why is the : 33.87
what color is the : 92.55
is there a : 97.61
is the : 96.52
is that a : 98.67
are these : 98.02
are the : 95.71
what is the : 78.51
which : 67.11
could : 97.83
are there : 98.11
what kind of : 79.44
has : 96.70
what color are the : 90.38
are there any : 98.15
is this person : 98.34
does the : 95.86
where is the : 53.13
how many people are : 71.66
can you : 95.07
what type of : 79.88
what is the color of the : 94.18
what is on the : 75.54
does this : 97.87
is this an : 99.15
who is : 75.79
what is this : 86.43
is there : 97.84
are : 94.13
what time : 27.90
how : 50.46
what is in the : 81.73
do you : 97.40
what are : 84.58
what sport is : 97.85
what is the person : 84.99
is he : 97.29
where are the :

In [12]:
# demo how to use evalQA to retrieve low score result
evals = [quesId for quesId in vqaEval.evalQA if vqaEval.evalQA[quesId]<35]   #35 is per question percentage accuracy
if len(evals) > 0:
	print ('ground truth answers')
	randomEval = random.choice(evals)
	randomAnn = vqa.loadQA(randomEval)
	vqa.showQA(randomAnn)

	print ('\n')
	print ('generated answer (accuracy %.02f)'%(vqaEval.evalQA[randomEval]))
	ann = vqaRes.loadQA(randomEval)[0]
	print ("Answer:   %s\n" %(ann['answer']))

	imgId = randomAnn[0]['image_id']
	imgFilename = 'COCO_' + dataSubType + '_'+ str(imgId).zfill(12) + '.jpg'
	if os.path.isfile(imgDir + imgFilename):
		I = io.imread(imgDir + imgFilename)
		plt.imshow(I)
		plt.axis('off')
		plt.show()

# save evaluation results to ./Results folder
json.dump(vqaEval.accuracy,     open(accuracyFile,     'w'))
json.dump(vqaEval.evalQA,       open(evalQAFile,       'w'))
json.dump(vqaEval.evalQuesType, open(evalQuesTypeFile, 'w'))
json.dump(vqaEval.evalAnsType,  open(evalAnsTypeFile,  'w'))

ground truth answers
Question: Where is the toilet and computer monitor?
Answer 1: garbage dump
Answer 2: on concrete
Answer 3: outside
Answer 4: outside
Answer 5: on ground
Answer 6: dump
Answer 7: trash
Answer 8: outside
Answer 9: in junk yard
Answer 10: ground


generated answer (accuracy 30.00)
Answer:   on ground

